In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
import pyswmm
from pyswmm import Simulation, LidControls, LidGroups, Subcatchments
import os
import swmmio as sio

In [2]:

subbasin_attributes = pd.read_csv (r'C:\Users\R. Seth Herbst\Desktop\SWMM Programs and Files\PYTHON CSV files for MC\Target_MC_Subbasin_Data.csv')
subbasin_attributes = subbasin_attributes.set_index('Subbasin')


subbasin_attributes

,Areas,Perc_Imperv,...,GrassSwale_Standard_Surface_Area,Social_Context_Composite_Rank
Subbasin,,,,,
72591,930398.04,24.798,...,7460.877,4.175
72631,952047.36,43.946,...,10546.630,3.430
72671,1011681.00,29.089,...,8465.484,3.430
72731,954530.28,53.046,...,9221.800,3.430
72781,1649965.68,32.151,...,8414.588,3.912
72951,1785524.40,23.486,...,6868.850,3.912
73081,1531569.60,35.628,...,9263.514,3.430
73121,1481911.20,48.410,...,9874.122,3.430
73171,677488.68,57.681,...,11048.030,3.430


In [3]:
def individual_gen(subbasin_attributes):
    
        
    subbasin_attributes["RainGarden_Assigned_Areas"]=""
    subbasin_attributes["Bioretention_Assigned_Areas"]=""
    subbasin_attributes["GreenRoof_Assigned_Areas"]=""
    subbasin_attributes["PermeablePavement_Assigned_Areas"]=""
    subbasin_attributes["GrassSwale_Assigned_Areas"]=""
    

    perc_implementation = [0.25,0.5,0.75,1]

    for subbasin in subbasin_attributes.index:
        subbasin_attributes['RainGarden_Assigned_Areas'][subbasin] = subbasin_attributes['Feasible_RainGarden_Space'][subbasin] * random.choice(perc_implementation)

    for subbasin in subbasin_attributes.index:
        subbasin_attributes['Bioretention_Assigned_Areas'][subbasin] = subbasin_attributes['Feasible_Bioretention_Space'][subbasin] * random.choice(perc_implementation)
        
    for subbasin in subbasin_attributes.index:
        subbasin_attributes['GreenRoof_Assigned_Areas'][subbasin] = subbasin_attributes['Feasible_GreenRoof_Space'][subbasin] * random.choice(perc_implementation)
                    
    for subbasin in subbasin_attributes.index:
        subbasin_attributes['PermeablePavement_Assigned_Areas'][subbasin] = subbasin_attributes['Feasible_PermeablePavement_Space'][subbasin] * random.choice(perc_implementation)

    for subbasin in subbasin_attributes.index:
        subbasin_attributes['GrassSwale_Assigned_Areas'][subbasin] = subbasin_attributes['Feasible_GrassSwale_Space'][subbasin] * random.choice(perc_implementation)

        # ## Calculate total implemented LID areas for each subbasin


    subbasin_attributes["Total_LID_Area"] = ""

    for subbasin in subbasin_attributes.index:
        subbasin_attributes.at[subbasin,'Total_LID_Area'] = (subbasin_attributes.at[subbasin,'RainGarden_Assigned_Areas'] + 
                                                             subbasin_attributes.at[subbasin,'Bioretention_Assigned_Areas']+
                                                             subbasin_attributes.at[subbasin,'GreenRoof_Assigned_Areas'] +
                                                             subbasin_attributes.at[subbasin,'PermeablePavement_Assigned_Areas']+
                                                             subbasin_attributes.at[subbasin,'GrassSwale_Assigned_Areas'])


# ## Update impervious LID areas for each subbasin after LID implementation

    subbasin_attributes['Updated_Imp_Areas']=""

    for subbasin in subbasin_attributes.index:
        subbasin_attributes.at[subbasin,'Updated_Imp_Areas']=(subbasin_attributes.at[subbasin,'Imperv_Area'] -
                                                              subbasin_attributes.at[subbasin,'GreenRoof_Assigned_Areas']-
                                                              subbasin_attributes.at[subbasin,'PermeablePavement_Assigned_Areas'])
    # # Permeable Pavement LID Usage Editor Parameter Changes!
    # ## Permeable Pavement Percent Impervious Treated Calculation for each Subbasin



    subbasin_attributes['PermeablePavement_%ImpArea_Treated']=""

    for subbasin in subbasin_attributes.index:
        subbasin_attributes.at[subbasin,'PermeablePavement_%ImpArea_Treated'] = ((subbasin_attributes.at[subbasin,'PermeablePavement_Assigned_Areas']*2.5/
                                                                                  subbasin_attributes.at[subbasin,'Updated_Imp_Areas'])*100)
    
    subbasin_attributes.at[73461,'PermeablePavement_%ImpArea_Treated']    


# # Percent Impervious area and Percent Pervious area treated changes according to total LID area implemented
# # Following blocks calculate updated percent impervious area treated and percent impervious  changes for each subbasin based on LID scenario
# 

# ## RainGarden LID Usage Editor Parameter Changes!

# ### Calculations for RainGarden Percent Impervious Area Treated for each Subbasin

# In[9]:


    subbasin_attributes['RainGarden_Total_Units']=""

    for subbasin in subbasin_attributes.index:
        subbasin_attributes.at[subbasin,'RainGarden_Total_Units'] = (subbasin_attributes.at[subbasin,'RainGarden_Assigned_Areas']/subbasin_attributes.at[subbasin,'RainGarden_Standard_Surface_Area'])   



    subbasin_attributes['RainGarden_DrainageArea_Treated']=""

    for subbasin in subbasin_attributes.index:
        subbasin_attributes.at[subbasin,'RainGarden_DrainageArea_Treated']= subbasin_attributes.at[subbasin,'RainGarden_Total_Units']*21780
 

    subbasin_attributes['RainGarden_ImpervArea_Treated']=""

    for subbasin in subbasin_attributes.index:
        subbasin_attributes.at[subbasin,'RainGarden_ImpervArea_Treated'] = subbasin_attributes.at[subbasin,'RainGarden_DrainageArea_Treated']* 0.25


    subbasin_attributes['RainGarden_%ImpArea_Treated']=""

    for subbasin in subbasin_attributes.index:
        subbasin_attributes.at[subbasin,'RainGarden_%ImpArea_Treated'] = (subbasin_attributes.at[subbasin,'RainGarden_ImpervArea_Treated']*100/subbasin_attributes.at[subbasin,'Updated_Imp_Areas'])



    for subbasin in subbasin_attributes.index:
        if subbasin_attributes.at[subbasin,'RainGarden_%ImpArea_Treated'] > 100:
            subbasin_attributes.at[subbasin,'RainGarden_%ImpArea_Treated'] = 100



# ### Calculations for RainGarden Percent Pervious Area Treated for each Subbasin


    subbasin_attributes['RainGarden_PervArea_Treated'] =""

    for subbasin in subbasin_attributes.index:
        subbasin_attributes.at[subbasin,'RainGarden_PervArea_Treated'] = subbasin_attributes.at[subbasin,'RainGarden_DrainageArea_Treated'] * 0.75


    subbasin_attributes['RainGarden_%PervArea_Treated']=""
    for subbasin in subbasin_attributes.index:
        subbasin_attributes.at[subbasin,'RainGarden_%PervArea_Treated'] = 100*subbasin_attributes.at[subbasin,'RainGarden_PervArea_Treated']/subbasin_attributes.at[subbasin,'Perv_Area']  


    for subbasin in subbasin_attributes.index:
        if subbasin_attributes.at[subbasin,'RainGarden_%PervArea_Treated'] > 100:
            subbasin_attributes.at[subbasin,'RainGarden_%PervArea_Treated'] = 100


# ## Bioretention LID Usage Editor Parameter Changes!

# ### Percent of Impervious Area Treated Calculations for each Subbasin


    subbasin_attributes['Bioretention_Total_Units']=""

    for subbasin in subbasin_attributes.index:
        subbasin_attributes.at[subbasin,'Bioretention_Total_Units'] = (subbasin_attributes.at[subbasin,'Bioretention_Assigned_Areas']/subbasin_attributes.at[subbasin,'Bioretention_Standard_Surface_Area'])   


    subbasin_attributes['Bioretention_DrainageArea_Treated']=""

    for subbasin in subbasin_attributes.index:
        subbasin_attributes.at[subbasin,'Bioretention_DrainageArea_Treated']= subbasin_attributes.at[subbasin,'Bioretention_Total_Units']*163350   


    subbasin_attributes['Bioretention_ImpervArea_Treated']=""

    for subbasin in subbasin_attributes.index:
        subbasin_attributes.at[subbasin,'Bioretention_ImpervArea_Treated'] = subbasin_attributes.at[subbasin,'Bioretention_DrainageArea_Treated']* 0.5


    subbasin_attributes['Bioretention_%ImpArea_Treated']=""

    for subbasin in subbasin_attributes.index:
        subbasin_attributes.at[subbasin,'Bioretention_%ImpArea_Treated'] = (subbasin_attributes.at[subbasin,'Bioretention_ImpervArea_Treated']*100/subbasin_attributes.at[subbasin,'Updated_Imp_Areas'])

    for subbasin in subbasin_attributes.index:
        if subbasin_attributes.at[subbasin,'Bioretention_%ImpArea_Treated'] > 100:
            subbasin_attributes.at[subbasin,'Bioretention_%ImpArea_Treated'] = 100


# ### Bioretention Percent Pervious Area Treated for each Subbasin


    subbasin_attributes['Bioretention_PervArea_Treated'] =""

    for subbasin in subbasin_attributes.index:
        subbasin_attributes.at[subbasin,'Bioretention_PervArea_Treated'] = subbasin_attributes.at[subbasin,'Bioretention_DrainageArea_Treated'] * 0.5

    subbasin_attributes['Bioretention_%PervArea_Treated']=""
    
    for subbasin in subbasin_attributes.index:
        subbasin_attributes.at[subbasin,'Bioretention_%PervArea_Treated'] = 100*subbasin_attributes.at[subbasin,'Bioretention_PervArea_Treated']/subbasin_attributes.at[subbasin,'Perv_Area']
    

    for subbasin in subbasin_attributes.index:
        if subbasin_attributes.at[subbasin,'Bioretention_%PervArea_Treated'] > 100:
            subbasin_attributes.at[subbasin,'Bioretention_%PervArea_Treated'] = 100      


# ## Grass Swale LID Usage Editor Parameter Changes!

# ### Percent of Impervious Area Treated Calculations for each Subbasin



    subbasin_attributes['GrassSwale_Total_Units']=""

    for subbasin in subbasin_attributes.index:
        subbasin_attributes.at[subbasin,'GrassSwale_Total_Units'] = (subbasin_attributes.at[subbasin,'GrassSwale_Assigned_Areas']/subbasin_attributes.at[subbasin,'GrassSwale_Standard_Surface_Area'])   


    subbasin_attributes['GrassSwale_DrainageArea_Treated']=""

    for subbasin in subbasin_attributes.index:
        subbasin_attributes.at[subbasin,'GrassSwale_DrainageArea_Treated']= subbasin_attributes.at[subbasin,'GrassSwale_Total_Units']*163350   


    subbasin_attributes['GrassSwale_ImpervArea_Treated']=""

    for subbasin in subbasin_attributes.index:
        subbasin_attributes.at[subbasin,'GrassSwale_ImpervArea_Treated'] = subbasin_attributes.at[subbasin,'GrassSwale_DrainageArea_Treated']* 0.5


    subbasin_attributes['GrassSwale_%ImpArea_Treated']=""

    for subbasin in subbasin_attributes.index:
        subbasin_attributes.at[subbasin,'GrassSwale_%ImpArea_Treated'] = (subbasin_attributes.at[subbasin,'GrassSwale_ImpervArea_Treated']*100/subbasin_attributes.at[subbasin,'Updated_Imp_Areas'])



    for subbasin in subbasin_attributes.index:
        if subbasin_attributes.at[subbasin,'GrassSwale_%ImpArea_Treated'] > 100:
            subbasin_attributes.at[subbasin,'GrassSwale_%ImpArea_Treated'] = 100


# ### Percent of pervious Area Treated Calculations for each Subbasin


    subbasin_attributes['GrassSwale_PervArea_Treated'] =""

    for subbasin in subbasin_attributes.index:
        subbasin_attributes.at[subbasin,'GrassSwale_PervArea_Treated'] = subbasin_attributes.at[subbasin,'GrassSwale_DrainageArea_Treated'] * 0.5

        
    subbasin_attributes['GrassSwale_%PervArea_Treated']=""
    for subbasin in subbasin_attributes.index:
        subbasin_attributes.at[subbasin,'GrassSwale_%PervArea_Treated'] = 100*subbasin_attributes.at[subbasin,'GrassSwale_PervArea_Treated']/subbasin_attributes.at[subbasin,'Perv_Area']   


    for subbasin in subbasin_attributes.index:
        if subbasin_attributes.at[subbasin,'GrassSwale_%PervArea_Treated'] > 100:
            subbasin_attributes.at[subbasin,'GrassSwale_%PervArea_Treated'] = 100    


# ## Capping total %impervious and %pervious area treated at 100% for each subcatchment-LID combo. 


    subbasin_attributes['%Imp_Treated']=""

    for subbasin in subbasin_attributes.index:
        subbasin_attributes.at[subbasin,'%Imp_Treated'] = (subbasin_attributes.at[subbasin,'RainGarden_%ImpArea_Treated'] + 
                                                           subbasin_attributes.at[subbasin,'Bioretention_%ImpArea_Treated'] +
                                                           subbasin_attributes.at[subbasin,'PermeablePavement_%ImpArea_Treated']+
                                                           subbasin_attributes.at[subbasin,'GrassSwale_%ImpArea_Treated'])


    subbasin_attributes['%Perv_Treated']=""

    for subbasin in subbasin_attributes.index:
        subbasin_attributes.at[subbasin,'%Perv_Treated']= (subbasin_attributes.at[subbasin,'RainGarden_%PervArea_Treated']+
                                                           subbasin_attributes.at[subbasin,'Bioretention_%PervArea_Treated']+
                                                           subbasin_attributes.at[subbasin,'GrassSwale_%PervArea_Treated'])


# Pervious Surface Treated "Shaving"

    for subbasin in subbasin_attributes.index:

        while subbasin_attributes.at[subbasin,'%Perv_Treated'] > 100:
            subbasin_attributes.at[subbasin,'RainGarden_%PervArea_Treated']= subbasin_attributes.at[subbasin,'RainGarden_%PervArea_Treated']-1
            subbasin_attributes.at[subbasin,'Bioretention_%PervArea_Treated']=subbasin_attributes.at[subbasin,'Bioretention_%PervArea_Treated']-0.25
            subbasin_attributes.at[subbasin,'GrassSwale_%PervArea_Treated']=subbasin_attributes.at[subbasin,'GrassSwale_%PervArea_Treated']-0.25
        
            if subbasin_attributes.at[subbasin,'RainGarden_%PervArea_Treated'] < 0:
                subbasin_attributes.at[subbasin,'RainGarden_%PervArea_Treated'] = 0
            if subbasin_attributes.at[subbasin,'Bioretention_%PervArea_Treated'] < 0:
                subbasin_attributes.at[subbasin,'Bioretention_%PervArea_Treated'] = 0
            if subbasin_attributes.at[subbasin,'GrassSwale_%PervArea_Treated'] < 0:
                subbasin_attributes.at[subbasin,'GrassSwale_%PervArea_Treated'] = 0
        
            subbasin_attributes.at[subbasin, '%Perv_Treated'] = (subbasin_attributes.at[subbasin,'RainGarden_%PervArea_Treated']+
                                                                 subbasin_attributes.at[subbasin,'Bioretention_%PervArea_Treated']+
                                                                 subbasin_attributes.at[subbasin,'GrassSwale_%PervArea_Treated'])


# Impervious Surface Treated "Shaving"

    for subbasin in subbasin_attributes.index:
    
        while subbasin_attributes.at[subbasin,'%Imp_Treated'] > 100:
            subbasin_attributes.at[subbasin,'RainGarden_%ImpArea_Treated']=subbasin_attributes.at[subbasin,'RainGarden_%ImpArea_Treated']-1
            subbasin_attributes.at[subbasin,'Bioretention_%ImpArea_Treated']=subbasin_attributes.at[subbasin,'Bioretention_%ImpArea_Treated']-0.25
            subbasin_attributes.at[subbasin,'PermeablePavement_%ImpArea_Treated']=subbasin_attributes.at[subbasin,'PermeablePavement_%ImpArea_Treated']-0.25
            subbasin_attributes.at[subbasin,'GrassSwale_%ImpArea_Treated']=subbasin_attributes.at[subbasin,'GrassSwale_%ImpArea_Treated']-0.25        
        
        
            if subbasin_attributes.at[subbasin,'RainGarden_%ImpArea_Treated'] < 0:
                subbasin_attributes.at[subbasin,'RainGarden_%ImpArea_Treated'] = 0
            if subbasin_attributes.at[subbasin,'Bioretention_%ImpArea_Treated'] < 0:
                subbasin_attributes.at[subbasin,'Bioretention_%ImpArea_Treated'] = 0
            if subbasin_attributes.at[subbasin,'PermeablePavement_%ImpArea_Treated'] < 0:
                subbasin_attributes.at[subbasin,'PermeablePavement_%ImpArea_Treated'] = 0
            if subbasin_attributes.at[subbasin,'GrassSwale_%ImpArea_Treated'] < 0:
                subbasin_attributes.at[subbasin,'GrassSwale_%ImpArea_Treated'] = 0
        
        
        
        
            subbasin_attributes.at[subbasin, '%Imp_Treated'] = (subbasin_attributes.at[subbasin,'RainGarden_%ImpArea_Treated']+
                                                                subbasin_attributes.at[subbasin,'Bioretention_%ImpArea_Treated']+
                                                                subbasin_attributes.at[subbasin,'PermeablePavement_%ImpArea_Treated']+
                                                                subbasin_attributes.at[subbasin,'GrassSwale_%ImpArea_Treated'])

    subbasin_attributes['Updated_%Imp']=""

    for subbasin in subbasin_attributes.index:
        subbasin_attributes.at[subbasin,'Updated_%Imp'] = subbasin_attributes.at[subbasin,'Updated_Imp_Areas']*100 / subbasin_attributes.at[subbasin,'Areas']
    

#calculating costs for different LID scenarios

    subbasin_attributes["PermeablePavement_Costs"]=""

#assuming pervious concrete, installation ONLY - O&M unclear/makes price too high

    for subbasin in subbasin_attributes.index:
         subbasin_attributes.at[subbasin,'PermeablePavement_Costs'] = subbasin_attributes.at[subbasin,'PermeablePavement_Assigned_Areas'] * 9.79

        
    subbasin_attributes["GreenRoof_Costs"]=""

    for subbasin in subbasin_attributes.index:
        subbasin_attributes.at[subbasin,'GreenRoof_Costs'] = subbasin_attributes.at[subbasin,'GreenRoof_Assigned_Areas'] * 11.66        


# # Calculating Rain Garden Costs

    subbasin_attributes["Updated_RainGarden_ImpArea_Treated"] = ""
    for subbasin in subbasin_attributes.index:
        subbasin_attributes.at[subbasin,'Updated_RainGarden_ImpArea_Treated'] = (subbasin_attributes.at[subbasin,'RainGarden_%ImpArea_Treated']/100) * subbasin_attributes.at[subbasin,'Updated_Imp_Areas']
    
    
    subbasin_attributes['Updated_RainGarden_ImpArea_Treated']    
    

    subbasin_attributes['RainGarden_Costs']=""

    for subbasin in subbasin_attributes.index:
        subbasin_attributes.at[subbasin,'RainGarden_Costs'] = subbasin_attributes.at[subbasin,'Updated_RainGarden_ImpArea_Treated'] * 2.34

# # Calculating Bioretention Costs

    subbasin_attributes["Updated_Bioretention_ImpArea_Treated"] = ""
    for subbasin in subbasin_attributes.index:
        subbasin_attributes.at[subbasin,'Updated_Bioretention_ImpArea_Treated'] = (subbasin_attributes.at[subbasin,'Bioretention_%ImpArea_Treated']/100) * subbasin_attributes.at[subbasin,'Updated_Imp_Areas']
    
    
    subbasin_attributes['Updated_Bioretention_ImpArea_Treated']  


    subbasin_attributes['Bioretention_Costs']=""

    for subbasin in subbasin_attributes.index:
        subbasin_attributes.at[subbasin,'Bioretention_Costs'] = 2.34 * subbasin_attributes.at[subbasin,'Updated_Bioretention_ImpArea_Treated']    


# # Calculating Grass Swale Costs


    subbasin_attributes["Updated_GrassSwale_ImpArea_Treated"] = ""
    for subbasin in subbasin_attributes.index:
        subbasin_attributes.at[subbasin,'Updated_GrassSwale_ImpArea_Treated'] = (subbasin_attributes.at[subbasin,'GrassSwale_%ImpArea_Treated']/100) * subbasin_attributes.at[subbasin,'Updated_Imp_Areas']
    

    subbasin_attributes['GrassSwale_Costs']=""

    for subbasin in subbasin_attributes.index:
        subbasin_attributes.at[subbasin,'GrassSwale_Costs'] = 1.45 * subbasin_attributes.at[subbasin,'Updated_GrassSwale_ImpArea_Treated']
        
    
    subbasin_attributes['Subbasin_Total_LID_Costs']=""
    
    for subbasin in subbasin_attributes.index:
        subbasin_attributes.at[subbasin,'Subbasin_Total_LID_Costs'] = (subbasin_attributes.at[subbasin,'Bioretention_Costs']+subbasin_attributes.at[subbasin,'GrassSwale_Costs']+subbasin_attributes.at[subbasin,'RainGarden_Costs']+subbasin_attributes.at[subbasin,'GreenRoof_Costs'] + subbasin_attributes.at[subbasin,'PermeablePavement_Costs'])
        
    #Social Context Rank Calculations for each Subbasin
    
    subbasin_attributes['Subbasin_Social_Context_Investment']=""
    
    for subbasin in subbasin_attributes.index:
        subbasin_attributes.at[subbasin,'Subbasin_Social_Context_Investment'] = (subbasin_attributes.at[subbasin,'Social_Context_Composite_Rank']/subbasin_attributes.at[subbasin,'Subbasin_Total_LID_Costs'])
    
    
    return subbasin_attributes


In [4]:
memberrollcall = {}
def pop_gen(sb):
    
    for i in range(6):
        member = individual_gen(sb)
        member.to_csv(r'C:\Users\R. Seth Herbst\Desktop\SWMM Programs and Files\PYTHON CSV files for MC\GA Parts\LID Scenario Population\member{}.csv'.format(i))
        member = member.to_dict(orient='dict')
        memberrollcall[i]= member
        memberrollcall['member{}'.format(i)] = memberrollcall[i]
        del memberrollcall[i]       

In [5]:
pop_gen(subbasin_attributes)
memberrollcall

<ipython-input-3-20dfc3ba0f42>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subbasin_attributes['RainGarden_Assigned_Areas'][subbasin] = subbasin_attributes['Feasible_RainGarden_Space'][subbasin] * random.choice(perc_implementation)
<ipython-input-3-20dfc3ba0f42>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subbasin_attributes['Bioretention_Assigned_Areas'][subbasin] = subbasin_attributes['Feasible_Bioretention_Space'][subbasin] * random.choice(perc_implementation)
<ipython-input-3-20dfc3ba0f42>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in 

{'member0': {'Areas': {72591: 930398.04,
   72631: 952047.36,
   72671: 1011681.0,
   72731: 954530.28,
   72781: 1649965.68,
   72951: 1785524.4,
   73081: 1531569.6,
   73121: 1481911.2,
   73171: 677488.68,
   73181: 626044.32,
   73201: 922731.48,
   73211: 1332718.2,
   73301: 1379980.8,
   73321: 629311.32,
   73391: 762430.68,
   73421: 672827.76,
   73461: 708067.8,
   73531: 677662.92,
   73551: 603218.88,
   73571: 368648.28},
  'Perc_Imperv': {72591: 24.798000000000002,
   72631: 43.946000000000005,
   72671: 29.089000000000002,
   72731: 53.04600000000001,
   72781: 32.150999999999996,
   72951: 23.486,
   73081: 35.628,
   73121: 48.41,
   73171: 57.681000000000004,
   73181: 68.87100000000001,
   73201: 53.623999999999995,
   73211: 32.347,
   73301: 45.839,
   73321: 69.44800000000001,
   73391: 21.519000000000002,
   73421: 74.942,
   73461: 75.242,
   73531: 69.352,
   73551: 68.462,
   73571: 68.637},
  'FlowLengths': {72591: 1094.586,
   72631: 793.373,
   72671: 144

In [6]:
member_cost = {}

def total_costs_calculator(memberrollcall):
    
    for i in range(6):
        member_cost[i] = sum((memberrollcall['member{}'.format(i)]['Subbasin_Total_LID_Costs'].values()))
    return member_cost 

In [7]:
member_social_rank = {}

def social_context_investment_func(memberrollcall):
    for i in range(6):
        member_social_rank[i]=sum((memberrollcall['member{}'.format(i)]['Subbasin_Social_Context_Investment'].values()))
    
    return member_social_rank
    

In [8]:
# Population Fitness Member Scores

def Member_Fitness_Calculator():
    
    scenario_list = ['member0', 'member1', 'member2','member3','member4','member5']

    popmembers_df = pd.DataFrame(scenario_list, columns = ['Population_Members'])
    peak_flow_df = pd.DataFrame(peak_flows, index = ['Peak_Flow (CFS)']).T
    popmembers_df['Peak_Flow (CFS)'] = peak_flow_df['Peak_Flow (CFS)']

    total_runoff_volumes_df = pd.DataFrame(total_runoff_volumes, index = ['Total_Runoff_Volume (AF)']).T
    total_runoff_volumes_df['Population_Members'] = popmembers_df['Population_Members']

    total_costs_df = pd.DataFrame(total_costs,index = ['Total_Costs $']).T
    total_costs_df['Population_Members'] = popmembers_df['Population_Members']

    social_context_scores_df = pd.DataFrame(social_context_scores, index = ['Social_Context_Score']).T
    social_context_scores_df['Population_Members'] = popmembers_df['Population_Members']

    result = pd.merge(popmembers_df, total_runoff_volumes_df , on="Population_Members")
    result = pd.merge(result, total_costs_df, on = "Population_Members")
    population_fitness_metrics = pd.merge(result,social_context_scores_df, on = "Population_Members")
    #population_fitness_metrics = population_fitness_metrics.set_index('Population_Members')

    return population_fitness_metrics

In [9]:
total_costs = total_costs_calculator(memberrollcall)

social_context_scores = social_context_investment_func(memberrollcall)

In [10]:
def Population_Elites():


    winning_numbers = {'Peak Flow (CFS)': 99999, 'Total Runoff Volume (AF)': 99999, 'Total Costs ($)':999999999999999, 'Social Context Score': 99999999}
    winning_members = {'Peak Flow':None, 'Total Runoff': None, 'Total Costs': None, 'Social Context':None}
    for index, row in population_fitness_metrics.iterrows():

        if winning_numbers['Peak Flow (CFS)'] > population_fitness_metrics.iloc[[index]].at[index,'Peak_Flow (CFS)']:
            winning_numbers['Peak Flow (CFS)'] = population_fitness_metrics.iloc[[index]].at[index,'Peak_Flow (CFS)']
            winning_members['Peak Flow'] = population_fitness_metrics.iloc[[index]].at[index,'Population_Members']


        if winning_numbers['Total Runoff Volume (AF)'] > population_fitness_metrics.iloc[[index]].at[index,'Total_Runoff_Volume (AF)']:
            winning_numbers['Total Runoff Volume (AF)'] = population_fitness_metrics.iloc[[index]].at[index,'Total_Runoff_Volume (AF)']
            winning_members['Total Runoff'] = population_fitness_metrics.iloc[[index]].at[index,'Population_Members']


        if winning_numbers['Total Costs ($)'] > population_fitness_metrics.iloc[[index]].at[index,'Total_Costs $']:
            winning_numbers['Total Costs ($)'] = population_fitness_metrics.iloc[[index]].at[index,'Total_Costs $']
            winning_members['Total Costs'] = population_fitness_metrics.iloc[[index]].at[index,'Population_Members']

        if winning_numbers['Social Context Score'] > population_fitness_metrics.iloc[[index]].at[index,'Social_Context_Score']:
            winning_numbers['Social Context Score'] = population_fitness_metrics.iloc[[index]].at[index,'Social_Context_Score']
            winning_members['Social Context'] = population_fitness_metrics.iloc[[index]].at[index,'Population_Members']


       
    return winning_members

In [11]:
peak_flows = {}
total_runoff_volumes = {}

def swmm_model():
    

    
    for file in range(6):
        subbasin_attributes =pd.read_csv(r"C:\Users\R. Seth Herbst\Desktop\SWMM Programs and Files\PYTHON CSV files for MC\GA Parts\LID Scenario Population\member{}.csv".format(file))
        
        subbasin_attributes = subbasin_attributes.set_index('Subbasin')
        
        #initialize a baseline model object in swmmio
        baseline = sio.Model(r'C:\Users\R. Seth Herbst\Desktop\SWMM Programs and Files\MeadowCreekSWMM')

        #isolate subcatchments dataframe
        subcatchments = baseline.inp.subcatchments

        #Set subcatchment parameters

        #Percent Impervious Adjustments

        subcatchments.loc['73081','PercImperv'] = subbasin_attributes.at[73081,'Updated_%Imp']
        subcatchments.loc['72671','PercImperv'] = subbasin_attributes.at[72671,'Updated_%Imp']
        subcatchments.loc['72591','PercImperv'] = subbasin_attributes.at[72591,'Updated_%Imp']
        subcatchments.loc['72631','PercImperv'] = subbasin_attributes.at[72631,'Updated_%Imp']
        subcatchments.loc['72781','PercImperv'] = subbasin_attributes.at[72781,'Updated_%Imp']
        subcatchments.loc['72731','PercImperv'] = subbasin_attributes.at[72731,'Updated_%Imp']
        subcatchments.loc['72951','PercImperv'] = subbasin_attributes.at[72951,'Updated_%Imp']
        subcatchments.loc['73211','PercImperv'] = subbasin_attributes.at[73211,'Updated_%Imp']
        subcatchments.loc['73181','PercImperv'] = subbasin_attributes.at[73181,'Updated_%Imp']
        subcatchments.loc['73171','PercImperv'] = subbasin_attributes.at[73171,'Updated_%Imp']
        subcatchments.loc['73121','PercImperv'] = subbasin_attributes.at[73121,'Updated_%Imp']
        subcatchments.loc['73201','PercImperv'] = subbasin_attributes.at[73201,'Updated_%Imp']
        subcatchments.loc['73301','PercImperv'] = subbasin_attributes.at[73301,'Updated_%Imp']
        subcatchments.loc['73321','PercImperv'] = subbasin_attributes.at[73321,'Updated_%Imp']
        subcatchments.loc['73391','PercImperv'] = subbasin_attributes.at[73391,'Updated_%Imp']
        subcatchments.loc['73461','PercImperv'] = subbasin_attributes.at[73461,'Updated_%Imp']
        subcatchments.loc['73551','PercImperv'] = subbasin_attributes.at[73551,'Updated_%Imp']
        subcatchments.loc['73421','PercImperv'] = subbasin_attributes.at[73421,'Updated_%Imp']
        subcatchments.loc['73531','PercImperv'] = subbasin_attributes.at[73531,'Updated_%Imp']
        subcatchments.loc['73571','PercImperv'] = subbasin_attributes.at[73571,'Updated_%Imp']

        #Width Adjustment((SUBBASIN AREA/Flow Length) - LID Width)

        subcatchments.loc['73081','Width'] = ((subcatchments.loc['73081','Area'] * 43560)/subbasin_attributes.at[73081,'FlowLengths']) - (subbasin_attributes.at[73081,'Total_LID_Area']/subbasin_attributes.at[73081,'FlowLengths'])
        subcatchments.loc['72671','Width'] = ((subcatchments.loc['72671','Area'] * 43560)/subbasin_attributes.at[72671,'FlowLengths']) - (subbasin_attributes.at[72671,'Total_LID_Area']/subbasin_attributes.at[72671,'FlowLengths'])
        subcatchments.loc['72591','Width'] = ((subcatchments.loc['72591','Area'] * 43560)/subbasin_attributes.at[72591,'FlowLengths']) - (subbasin_attributes.at[72591,'Total_LID_Area']/subbasin_attributes.at[72591,'FlowLengths'])
        subcatchments.loc['72631','Width'] = ((subcatchments.loc['72631','Area'] * 43560)/subbasin_attributes.at[72631,'FlowLengths']) - (subbasin_attributes.at[72631,'Total_LID_Area']/subbasin_attributes.at[72631,'FlowLengths'])
        subcatchments.loc['72781','Width'] = ((subcatchments.loc['72781','Area'] * 43560)/subbasin_attributes.at[72781,'FlowLengths']) - (subbasin_attributes.at[72781,'Total_LID_Area']/subbasin_attributes.at[72781,'FlowLengths'])
        subcatchments.loc['72731','Width'] = ((subcatchments.loc['72731','Area'] * 43560)/subbasin_attributes.at[72731,'FlowLengths']) - (subbasin_attributes.at[72731,'Total_LID_Area']/subbasin_attributes.at[72731,'FlowLengths'])
        subcatchments.loc['72951','Width'] = ((subcatchments.loc['72951','Area'] * 43560)/subbasin_attributes.at[72951,'FlowLengths']) - (subbasin_attributes.at[72951,'Total_LID_Area']/subbasin_attributes.at[72951,'FlowLengths'])
        subcatchments.loc['73211','Width'] = ((subcatchments.loc['73211','Area'] * 43560)/subbasin_attributes.at[73211,'FlowLengths']) - (subbasin_attributes.at[73211,'Total_LID_Area']/subbasin_attributes.at[73211,'FlowLengths'])
        subcatchments.loc['73181','Width'] = ((subcatchments.loc['73181','Area'] * 43560)/subbasin_attributes.at[73181,'FlowLengths']) - (subbasin_attributes.at[73181,'Total_LID_Area']/subbasin_attributes.at[73181,'FlowLengths'])
        subcatchments.loc['73171','Width'] = ((subcatchments.loc['73171','Area'] * 43560)/subbasin_attributes.at[73171,'FlowLengths']) - (subbasin_attributes.at[73171,'Total_LID_Area']/subbasin_attributes.at[73171,'FlowLengths'])
        subcatchments.loc['73121','Width'] = ((subcatchments.loc['73121','Area'] * 43560)/subbasin_attributes.at[73121,'FlowLengths']) - (subbasin_attributes.at[73121,'Total_LID_Area']/subbasin_attributes.at[73121,'FlowLengths'])
        subcatchments.loc['73201','Width'] = ((subcatchments.loc['73201','Area'] * 43560)/subbasin_attributes.at[73201,'FlowLengths']) - (subbasin_attributes.at[73201,'Total_LID_Area']/subbasin_attributes.at[73201,'FlowLengths'])
        subcatchments.loc['73301','Width'] = ((subcatchments.loc['73301','Area'] * 43560)/subbasin_attributes.at[73301,'FlowLengths']) - (subbasin_attributes.at[73301,'Total_LID_Area']/subbasin_attributes.at[73301,'FlowLengths'])
        subcatchments.loc['73321','Width'] = ((subcatchments.loc['73321','Area'] * 43560)/subbasin_attributes.at[73321,'FlowLengths']) - (subbasin_attributes.at[73321,'Total_LID_Area']/subbasin_attributes.at[73321,'FlowLengths'])
        subcatchments.loc['73391','Width'] = ((subcatchments.loc['73391','Area'] * 43560)/subbasin_attributes.at[73391,'FlowLengths']) - (subbasin_attributes.at[73391,'Total_LID_Area']/subbasin_attributes.at[73391,'FlowLengths'])
        subcatchments.loc['73461','Width'] = ((subcatchments.loc['73461','Area'] * 43560)/subbasin_attributes.at[73461,'FlowLengths']) - (subbasin_attributes.at[73461,'Total_LID_Area']/subbasin_attributes.at[73461,'FlowLengths'])
        subcatchments.loc['73551','Width'] = ((subcatchments.loc['73551','Area'] * 43560)/subbasin_attributes.at[73551,'FlowLengths']) - (subbasin_attributes.at[73551,'Total_LID_Area']/subbasin_attributes.at[73551,'FlowLengths'])
        subcatchments.loc['73421','Width'] = ((subcatchments.loc['73421','Area'] * 43560)/subbasin_attributes.at[73421,'FlowLengths']) - (subbasin_attributes.at[73421,'Total_LID_Area']/subbasin_attributes.at[73421,'FlowLengths'])
        subcatchments.loc['73531','Width'] = ((subcatchments.loc['73531','Area'] * 43560)/subbasin_attributes.at[73531,'FlowLengths']) - (subbasin_attributes.at[73531,'Total_LID_Area']/subbasin_attributes.at[73531,'FlowLengths'])
        subcatchments.loc['73571','Width'] = ((subcatchments.loc['73571','Area'] * 43560)/subbasin_attributes.at[73571,'FlowLengths']) - (subbasin_attributes.at[73571,'Total_LID_Area']/subbasin_attributes.at[73571,'FlowLengths'])

        #save subcatchment dataframe changes to whole inp dataframe
        baseline.inp.subcatchments = subcatchments

        #create the new inp file to use. 
        newfilepath = os.path.join(r"C:\Users\R. Seth Herbst\Desktop\SWMM Programs and Files\PYTHON CSV files for MC\GA Parts\SWMMIO_Altered_INPUTS",'member{}.inp'.format(file))

        #Write the subcatchment section of the new model with the adjusted data

        baseline.inp.save(newfilepath)

        with Simulation(r"C:\Users\R. Seth Herbst\Desktop\SWMM Programs and Files\PYTHON CSV files for MC\GA Parts\SWMMIO_Altered_INPUTS\member{}.inp".format(file), reportfile = r'C:\Users\R. Seth Herbst\Desktop\SWMM Programs and Files\PYTHON CSV files for MC\GA Parts\SWMM_Results\member{}.txt'.format(file)) as sim:


            control_time_step = 86400*3
            sim.step_advance(control_time_step)


            #Load in subcatchment lid list

            lid_sub_73081 = LidGroups(sim)['73081']
            lid_sub_72671 = LidGroups(sim)['72671']
            lid_sub_72591 = LidGroups(sim)['72591']
            lid_sub_72631 = LidGroups(sim)['72631']
            lid_sub_72781 = LidGroups(sim)['72781']
            lid_sub_72731 = LidGroups(sim)['72731']
            lid_sub_72951 = LidGroups(sim)['72951']
            lid_sub_73211 = LidGroups(sim)['73211']
            lid_sub_73181 = LidGroups(sim)['73181']
            lid_sub_73171 = LidGroups(sim)['73171']
            lid_sub_73121 = LidGroups(sim)['73121']
            lid_sub_73201 = LidGroups(sim)['73201']
            lid_sub_73301 = LidGroups(sim)['73301']
            lid_sub_73321 = LidGroups(sim)['73321']
            lid_sub_73391 = LidGroups(sim)['73391']
            lid_sub_73461 = LidGroups(sim)['73461']
            lid_sub_73551 = LidGroups(sim)['73551']
            lid_sub_73421 = LidGroups(sim)['73421']
            lid_sub_73531 = LidGroups(sim)['73531']
            lid_sub_73571 = LidGroups(sim)['73571']


        #     #Set LID Areas!


            lid_sub_73081[0].unit_area = subbasin_attributes.at[73081,'GreenRoof_Assigned_Areas']
            lid_sub_73081[1].unit_area = subbasin_attributes.at[73081,'PermeablePavement_Assigned_Areas']
            lid_sub_73081[2].unit_area = subbasin_attributes.at[73081,'RainGarden_Assigned_Areas']
            lid_sub_73081[3].unit_area = subbasin_attributes.at[73081,'Bioretention_Assigned_Areas']
            lid_sub_73081[4].unit_area = subbasin_attributes.at[73081,'GrassSwale_Assigned_Areas']

            lid_sub_72671[0].unit_area = subbasin_attributes.at[72671,'GreenRoof_Assigned_Areas']
            lid_sub_72671[1].unit_area = subbasin_attributes.at[72671,'PermeablePavement_Assigned_Areas']
            lid_sub_72671[2].unit_area = subbasin_attributes.at[72671,'RainGarden_Assigned_Areas']
            lid_sub_72671[3].unit_area = subbasin_attributes.at[72671,'Bioretention_Assigned_Areas']
            lid_sub_72671[4].unit_area = subbasin_attributes.at[72671,'GrassSwale_Assigned_Areas']

            lid_sub_72591[0].unit_area = subbasin_attributes.at[72591,'GreenRoof_Assigned_Areas']
            lid_sub_72591[1].unit_area = subbasin_attributes.at[72591,'PermeablePavement_Assigned_Areas']
            lid_sub_72591[2].unit_area = subbasin_attributes.at[72591,'RainGarden_Assigned_Areas']
            lid_sub_72591[3].unit_area = subbasin_attributes.at[72591,'Bioretention_Assigned_Areas']
            lid_sub_72591[4].unit_area = subbasin_attributes.at[72591,'GrassSwale_Assigned_Areas']

            lid_sub_72631[0].unit_area = subbasin_attributes.at[72631,'GreenRoof_Assigned_Areas']
            lid_sub_72631[1].unit_area = subbasin_attributes.at[72631,'PermeablePavement_Assigned_Areas']
            lid_sub_72631[2].unit_area = subbasin_attributes.at[72631,'RainGarden_Assigned_Areas']
            lid_sub_72631[3].unit_area = subbasin_attributes.at[72631,'Bioretention_Assigned_Areas']
            lid_sub_72631[4].unit_area = subbasin_attributes.at[72631,'GrassSwale_Assigned_Areas']

            lid_sub_72781[0].unit_area = subbasin_attributes.at[72781,'GreenRoof_Assigned_Areas']
            lid_sub_72781[1].unit_area = subbasin_attributes.at[72781,'PermeablePavement_Assigned_Areas']
            lid_sub_72781[2].unit_area = subbasin_attributes.at[72781,'RainGarden_Assigned_Areas']
            lid_sub_72781[3].unit_area = subbasin_attributes.at[72781,'Bioretention_Assigned_Areas']
            lid_sub_72781[4].unit_area = subbasin_attributes.at[72781,'GrassSwale_Assigned_Areas']

            lid_sub_72731[0].unit_area = subbasin_attributes.at[72731,'GreenRoof_Assigned_Areas']
            lid_sub_72731[1].unit_area = subbasin_attributes.at[72731,'PermeablePavement_Assigned_Areas']
            lid_sub_72731[2].unit_area = subbasin_attributes.at[72731,'RainGarden_Assigned_Areas']
            lid_sub_72731[3].unit_area = subbasin_attributes.at[72731,'Bioretention_Assigned_Areas']
            lid_sub_72731[4].unit_area = subbasin_attributes.at[72731,'GrassSwale_Assigned_Areas']

            lid_sub_72951[0].unit_area = subbasin_attributes.at[72951,'GreenRoof_Assigned_Areas']
            lid_sub_72951[1].unit_area = subbasin_attributes.at[72951,'PermeablePavement_Assigned_Areas']
            lid_sub_72951[2].unit_area = subbasin_attributes.at[72951,'RainGarden_Assigned_Areas']
            lid_sub_72951[3].unit_area = subbasin_attributes.at[72951,'PermeablePavement_Assigned_Areas']
            lid_sub_72951[4].unit_area = subbasin_attributes.at[72951,'GrassSwale_Assigned_Areas']

            lid_sub_73211[0].unit_area = subbasin_attributes.at[73211,'GreenRoof_Assigned_Areas']
            lid_sub_73211[1].unit_area = subbasin_attributes.at[73211,'PermeablePavement_Assigned_Areas']
            lid_sub_73211[2].unit_area = subbasin_attributes.at[73211,'RainGarden_Assigned_Areas']
            lid_sub_73211[3].unit_area = subbasin_attributes.at[73211,'Bioretention_Assigned_Areas']
            lid_sub_73211[4].unit_area = subbasin_attributes.at[73211,'GrassSwale_Assigned_Areas']

            lid_sub_73181[0].unit_area = subbasin_attributes.at[73181,'GreenRoof_Assigned_Areas']
            lid_sub_73181[1].unit_area = subbasin_attributes.at[73181,'PermeablePavement_Assigned_Areas']
            lid_sub_73181[2].unit_area = subbasin_attributes.at[73181,'RainGarden_Assigned_Areas']
            lid_sub_73181[3].unit_area = subbasin_attributes.at[73181,'Bioretention_Assigned_Areas']
            lid_sub_73181[4].unit_area = subbasin_attributes.at[73181,'GrassSwale_Assigned_Areas']

            lid_sub_73171[0].unit_area = subbasin_attributes.at[73171,'GreenRoof_Assigned_Areas']
            lid_sub_73171[1].unit_area = subbasin_attributes.at[73171,'PermeablePavement_Assigned_Areas']
            lid_sub_73171[2].unit_area = subbasin_attributes.at[73171,'RainGarden_Assigned_Areas']
            lid_sub_73171[3].unit_area = subbasin_attributes.at[73171,'Bioretention_Assigned_Areas']
            lid_sub_73171[4].unit_area = subbasin_attributes.at[73171,'GrassSwale_Assigned_Areas']

            lid_sub_73121[0].unit_area = subbasin_attributes.at[73121,'GreenRoof_Assigned_Areas']
            lid_sub_73121[1].unit_area = subbasin_attributes.at[73121,'PermeablePavement_Assigned_Areas']
            lid_sub_73121[2].unit_area = subbasin_attributes.at[73121,'RainGarden_Assigned_Areas']
            lid_sub_73121[3].unit_area = subbasin_attributes.at[73121,'Bioretention_Assigned_Areas']
            lid_sub_73121[4].unit_area = subbasin_attributes.at[73121,'GrassSwale_Assigned_Areas']

            lid_sub_73201[0].unit_area = subbasin_attributes.at[73201,'GreenRoof_Assigned_Areas']
            lid_sub_73201[1].unit_area = subbasin_attributes.at[73201,'PermeablePavement_Assigned_Areas']
            lid_sub_73201[2].unit_area = subbasin_attributes.at[73201,'RainGarden_Assigned_Areas']
            lid_sub_73201[3].unit_area = subbasin_attributes.at[73201,'Bioretention_Assigned_Areas']
            lid_sub_73201[4].unit_area = subbasin_attributes.at[73201,'GrassSwale_Assigned_Areas']

            lid_sub_73301[0].unit_area = subbasin_attributes.at[73301,'GreenRoof_Assigned_Areas']
            lid_sub_73301[1].unit_area = subbasin_attributes.at[73301,'PermeablePavement_Assigned_Areas']
            lid_sub_73301[2].unit_area = subbasin_attributes.at[73301,'RainGarden_Assigned_Areas']
            lid_sub_73301[3].unit_area = subbasin_attributes.at[73301,'Bioretention_Assigned_Areas']
            lid_sub_73301[4].unit_area = subbasin_attributes.at[73301,'GrassSwale_Assigned_Areas']

            lid_sub_73321[0].unit_area = subbasin_attributes.at[73321,'GreenRoof_Assigned_Areas']
            lid_sub_73321[1].unit_area = subbasin_attributes.at[73321,'PermeablePavement_Assigned_Areas']
            lid_sub_73321[2].unit_area = subbasin_attributes.at[73321,'RainGarden_Assigned_Areas']
            lid_sub_73321[3].unit_area = subbasin_attributes.at[73321,'Bioretention_Assigned_Areas']
            lid_sub_73321[4].unit_area = subbasin_attributes.at[73321,'GrassSwale_Assigned_Areas']

            lid_sub_73391[0].unit_area = subbasin_attributes.at[73391,'GreenRoof_Assigned_Areas']
            lid_sub_73391[1].unit_area = subbasin_attributes.at[73391,'PermeablePavement_Assigned_Areas']
            lid_sub_73391[2].unit_area = subbasin_attributes.at[73391,'RainGarden_Assigned_Areas']
            lid_sub_73391[3].unit_area = subbasin_attributes.at[73391,'Bioretention_Assigned_Areas']
            lid_sub_73391[4].unit_area = subbasin_attributes.at[73391,'GrassSwale_Assigned_Areas']

            lid_sub_73461[0].unit_area = subbasin_attributes.at[73461,'GreenRoof_Assigned_Areas']
            lid_sub_73461[1].unit_area = subbasin_attributes.at[73461,'PermeablePavement_Assigned_Areas']
            lid_sub_73461[2].unit_area = subbasin_attributes.at[73461,'RainGarden_Assigned_Areas']
            lid_sub_73461[3].unit_area = subbasin_attributes.at[73461,'Bioretention_Assigned_Areas']
            lid_sub_73461[4].unit_area = subbasin_attributes.at[73461,'GrassSwale_Assigned_Areas']

            lid_sub_73551[0].unit_area = subbasin_attributes.at[73551,'GreenRoof_Assigned_Areas']
            lid_sub_73551[1].unit_area = subbasin_attributes.at[73551,'PermeablePavement_Assigned_Areas']
            lid_sub_73551[2].unit_area = subbasin_attributes.at[73551,'RainGarden_Assigned_Areas']
            lid_sub_73551[3].unit_area = subbasin_attributes.at[73551,'Bioretention_Assigned_Areas']
            lid_sub_73551[4].unit_area = subbasin_attributes.at[73551,'GrassSwale_Assigned_Areas']

            lid_sub_73421[0].unit_area = subbasin_attributes.at[73421,'GreenRoof_Assigned_Areas']
            lid_sub_73421[1].unit_area = subbasin_attributes.at[73421,'PermeablePavement_Assigned_Areas']
            lid_sub_73421[2].unit_area = subbasin_attributes.at[73421,'RainGarden_Assigned_Areas']
            lid_sub_73421[3].unit_area = subbasin_attributes.at[73421,'Bioretention_Assigned_Areas']
            lid_sub_73421[4].unit_area = subbasin_attributes.at[73421,'GrassSwale_Assigned_Areas']

            lid_sub_73531[0].unit_area = subbasin_attributes.at[73531,'GreenRoof_Assigned_Areas']
            lid_sub_73531[1].unit_area = subbasin_attributes.at[73531,'PermeablePavement_Assigned_Areas']
            lid_sub_73531[2].unit_area = subbasin_attributes.at[73531,'RainGarden_Assigned_Areas']
            lid_sub_73531[3].unit_area = subbasin_attributes.at[73531,'Bioretention_Assigned_Areas']
            lid_sub_73531[4].unit_area = subbasin_attributes.at[73531,'GrassSwale_Assigned_Areas']

            lid_sub_73571[0].unit_area = subbasin_attributes.at[73571,'GreenRoof_Assigned_Areas']
            lid_sub_73571[1].unit_area = subbasin_attributes.at[73571,'PermeablePavement_Assigned_Areas']
            lid_sub_73571[2].unit_area = subbasin_attributes.at[73571,'RainGarden_Assigned_Areas']
            lid_sub_73571[3].unit_area = subbasin_attributes.at[73571,'Bioretention_Assigned_Areas']
            lid_sub_73571[4].unit_area = subbasin_attributes.at[73571,'GrassSwale_Assigned_Areas']



        #Set Percent Impervious Areas Treated


            lid_sub_73081[1].from_impervious = subbasin_attributes.at[73081,'PermeablePavement_%ImpArea_Treated'] 
            lid_sub_73081[2].from_impervious = subbasin_attributes.at[73081,'RainGarden_%ImpArea_Treated'] 
            lid_sub_73081[3].from_impervious = subbasin_attributes.at[73081,'Bioretention_%ImpArea_Treated'] 
            lid_sub_73081[4].from_impervious = subbasin_attributes.at[73081,'GrassSwale_%ImpArea_Treated'] 

            lid_sub_72671[1].from_impervious = subbasin_attributes.at[72671,'PermeablePavement_%ImpArea_Treated'] 
            lid_sub_72671[2].from_impervious = subbasin_attributes.at[72671,'RainGarden_%ImpArea_Treated'] 
            lid_sub_72671[3].from_impervious = subbasin_attributes.at[72671,'Bioretention_%ImpArea_Treated'] 
            lid_sub_72671[4].from_impervious = subbasin_attributes.at[72671,'GrassSwale_%ImpArea_Treated'] 

            lid_sub_72591[1].from_impervious = subbasin_attributes.at[72591,'PermeablePavement_%ImpArea_Treated'] 
            lid_sub_72591[2].from_impervious = subbasin_attributes.at[72591,'RainGarden_%ImpArea_Treated'] 
            lid_sub_72591[3].from_impervious = subbasin_attributes.at[72591,'Bioretention_%ImpArea_Treated'] 
            lid_sub_72591[4].from_impervious = subbasin_attributes.at[72591,'GrassSwale_%ImpArea_Treated'] 

            lid_sub_72631[1].from_impervious = subbasin_attributes.at[72631,'PermeablePavement_%ImpArea_Treated'] 
            lid_sub_72631[2].from_impervious = subbasin_attributes.at[72631,'RainGarden_%ImpArea_Treated'] 
            lid_sub_72631[3].from_impervious = subbasin_attributes.at[72631,'Bioretention_%ImpArea_Treated'] 
            lid_sub_72631[4].from_impervious = subbasin_attributes.at[72631,'GrassSwale_%ImpArea_Treated'] 

            lid_sub_72781[1].from_impervious = subbasin_attributes.at[72781,'PermeablePavement_%ImpArea_Treated'] 
            lid_sub_72781[2].from_impervious = subbasin_attributes.at[72781,'RainGarden_%ImpArea_Treated'] 
            lid_sub_72781[3].from_impervious = subbasin_attributes.at[72781,'Bioretention_%ImpArea_Treated'] 
            lid_sub_72781[4].from_impervious = subbasin_attributes.at[72781,'GrassSwale_%ImpArea_Treated'] 

            lid_sub_72731[1].from_impervious = subbasin_attributes.at[72731,'PermeablePavement_%ImpArea_Treated'] 
            lid_sub_72731[2].from_impervious = subbasin_attributes.at[72731,'RainGarden_%ImpArea_Treated'] 
            lid_sub_72731[3].from_impervious = subbasin_attributes.at[72731,'Bioretention_%ImpArea_Treated'] 
            lid_sub_72731[4].from_impervious = subbasin_attributes.at[72731,'GrassSwale_%ImpArea_Treated'] 

            lid_sub_72951[1].from_impervious = subbasin_attributes.at[72951,'PermeablePavement_%ImpArea_Treated'] 
            lid_sub_72951[2].from_impervious = subbasin_attributes.at[72951,'RainGarden_%ImpArea_Treated'] 
            lid_sub_72951[3].from_impervious = subbasin_attributes.at[72951,'Bioretention_%ImpArea_Treated'] 
            lid_sub_72951[4].from_impervious = subbasin_attributes.at[72951,'GrassSwale_%ImpArea_Treated'] 


            lid_sub_73211[1].from_impervious = subbasin_attributes.at[73211,'PermeablePavement_%ImpArea_Treated'] 
            lid_sub_73211[2].from_impervious = subbasin_attributes.at[73211,'RainGarden_%ImpArea_Treated'] 
            lid_sub_73211[3].from_impervious = subbasin_attributes.at[73211,'Bioretention_%ImpArea_Treated'] 
            lid_sub_73211[4].from_impervious = subbasin_attributes.at[73211,'GrassSwale_%ImpArea_Treated'] 

            lid_sub_73181[1].from_impervious = subbasin_attributes.at[73181,'PermeablePavement_%ImpArea_Treated'] 
            lid_sub_73181[2].from_impervious = subbasin_attributes.at[73181,'RainGarden_%ImpArea_Treated'] 
            lid_sub_73181[3].from_impervious = subbasin_attributes.at[73181,'Bioretention_%ImpArea_Treated'] 
            lid_sub_73181[4].from_impervious = subbasin_attributes.at[73181,'GrassSwale_%ImpArea_Treated'] 

            lid_sub_73171[1].from_impervious = subbasin_attributes.at[73181,'PermeablePavement_%ImpArea_Treated'] 
            lid_sub_73171[2].from_impervious = subbasin_attributes.at[73181,'RainGarden_%ImpArea_Treated'] 
            lid_sub_73171[3].from_impervious = subbasin_attributes.at[73181,'Bioretention_%ImpArea_Treated'] 
            lid_sub_73171[4].from_impervious = subbasin_attributes.at[73181,'GrassSwale_%ImpArea_Treated'] 

            lid_sub_73121[1].from_impervious = subbasin_attributes.at[73121,'PermeablePavement_%ImpArea_Treated'] 
            lid_sub_73121[2].from_impervious = subbasin_attributes.at[73121,'RainGarden_%ImpArea_Treated'] 
            lid_sub_73121[3].from_impervious = subbasin_attributes.at[73121,'Bioretention_%ImpArea_Treated'] 
            lid_sub_73121[4].from_impervious = subbasin_attributes.at[73121,'GrassSwale_%ImpArea_Treated'] 

            lid_sub_73201[1].from_impervious = subbasin_attributes.at[73201,'PermeablePavement_%ImpArea_Treated'] 
            lid_sub_73201[2].from_impervious = subbasin_attributes.at[73201,'RainGarden_%ImpArea_Treated'] 
            lid_sub_73201[3].from_impervious = subbasin_attributes.at[73201,'Bioretention_%ImpArea_Treated'] 
            lid_sub_73201[4].from_impervious = subbasin_attributes.at[73201,'GrassSwale_%ImpArea_Treated'] 

            lid_sub_73301[1].from_impervious = subbasin_attributes.at[73301,'PermeablePavement_%ImpArea_Treated'] 
            lid_sub_73301[2].from_impervious = subbasin_attributes.at[73301,'RainGarden_%ImpArea_Treated'] 
            lid_sub_73301[3].from_impervious = subbasin_attributes.at[73301,'Bioretention_%ImpArea_Treated'] 
            lid_sub_73301[4].from_impervious = subbasin_attributes.at[73301,'GrassSwale_%ImpArea_Treated'] 

            lid_sub_73321[1].from_impervious = subbasin_attributes.at[73321,'PermeablePavement_%ImpArea_Treated'] 
            lid_sub_73321[2].from_impervious = subbasin_attributes.at[73321,'RainGarden_%ImpArea_Treated'] 
            lid_sub_73321[3].from_impervious = subbasin_attributes.at[73321,'Bioretention_%ImpArea_Treated'] 
            lid_sub_73321[4].from_impervious = subbasin_attributes.at[73321,'GrassSwale_%ImpArea_Treated'] 

            lid_sub_73391[1].from_impervious = subbasin_attributes.at[73391,'PermeablePavement_%ImpArea_Treated'] 
            lid_sub_73391[2].from_impervious = subbasin_attributes.at[73391,'RainGarden_%ImpArea_Treated'] 
            lid_sub_73391[3].from_impervious = subbasin_attributes.at[73391,'Bioretention_%ImpArea_Treated'] 
            lid_sub_73391[4].from_impervious = subbasin_attributes.at[73391,'GrassSwale_%ImpArea_Treated'] 

            lid_sub_73461[1].from_impervious = subbasin_attributes.at[73461,'PermeablePavement_%ImpArea_Treated'] 
            lid_sub_73461[2].from_impervious = subbasin_attributes.at[73461,'RainGarden_%ImpArea_Treated'] 
            lid_sub_73461[3].from_impervious = subbasin_attributes.at[73461,'Bioretention_%ImpArea_Treated'] 
            lid_sub_73461[4].from_impervious = subbasin_attributes.at[73461,'GrassSwale_%ImpArea_Treated'] 

            lid_sub_73551[1].from_impervious = subbasin_attributes.at[73551,'PermeablePavement_%ImpArea_Treated'] 
            lid_sub_73551[2].from_impervious = subbasin_attributes.at[73551,'RainGarden_%ImpArea_Treated'] 
            lid_sub_73551[3].from_impervious = subbasin_attributes.at[73551,'Bioretention_%ImpArea_Treated'] 
            lid_sub_73551[4].from_impervious = subbasin_attributes.at[73551,'GrassSwale_%ImpArea_Treated'] 

            lid_sub_73421[1].from_impervious = subbasin_attributes.at[73421,'PermeablePavement_%ImpArea_Treated'] 
            lid_sub_73421[2].from_impervious = subbasin_attributes.at[73421,'RainGarden_%ImpArea_Treated'] 
            lid_sub_73421[3].from_impervious = subbasin_attributes.at[73421,'Bioretention_%ImpArea_Treated'] 
            lid_sub_73421[4].from_impervious = subbasin_attributes.at[73421,'GrassSwale_%ImpArea_Treated'] 

            lid_sub_73531[1].from_impervious = subbasin_attributes.at[73531,'PermeablePavement_%ImpArea_Treated'] 
            lid_sub_73531[2].from_impervious = subbasin_attributes.at[73531,'RainGarden_%ImpArea_Treated'] 
            lid_sub_73531[3].from_impervious = subbasin_attributes.at[73531,'Bioretention_%ImpArea_Treated'] 
            lid_sub_73531[4].from_impervious = subbasin_attributes.at[73531,'GrassSwale_%ImpArea_Treated'] 

            lid_sub_73571[1].from_impervious = subbasin_attributes.at[73571,'PermeablePavement_%ImpArea_Treated'] 
            lid_sub_73571[2].from_impervious = subbasin_attributes.at[73571,'RainGarden_%ImpArea_Treated'] 
            lid_sub_73571[3].from_impervious = subbasin_attributes.at[73571,'Bioretention_%ImpArea_Treated'] 
            lid_sub_73571[4].from_impervious = subbasin_attributes.at[73571,'GrassSwale_%ImpArea_Treated'] 

        # Set % Pervious Areas Treated

            lid_sub_73081[2].from_pervious = subbasin_attributes.at[73081,'RainGarden_%PervArea_Treated']  
            lid_sub_73081[3].from_pervious = subbasin_attributes.at[73081,'Bioretention_%PervArea_Treated']  
            lid_sub_73081[4].from_pervious = subbasin_attributes.at[73081,'GrassSwale_%PervArea_Treated']  

            lid_sub_72671[2].from_pervious = subbasin_attributes.at[72671,'RainGarden_%PervArea_Treated']  
            lid_sub_72671[3].from_pervious = subbasin_attributes.at[72671,'Bioretention_%PervArea_Treated']  
            lid_sub_72671[4].from_pervious = subbasin_attributes.at[72671,'GrassSwale_%PervArea_Treated']  

            lid_sub_72591[2].from_pervious = subbasin_attributes.at[72591,'RainGarden_%PervArea_Treated']  
            lid_sub_72591[3].from_pervious = subbasin_attributes.at[72591,'Bioretention_%PervArea_Treated']  
            lid_sub_72591[4].from_pervious = subbasin_attributes.at[72591,'GrassSwale_%PervArea_Treated']  

            lid_sub_72631[2].from_pervious = subbasin_attributes.at[72631,'RainGarden_%PervArea_Treated']  
            lid_sub_72631[3].from_pervious = subbasin_attributes.at[72631,'Bioretention_%PervArea_Treated']  
            lid_sub_72631[4].from_pervious = subbasin_attributes.at[72631,'GrassSwale_%PervArea_Treated']  

            lid_sub_72781[2].from_pervious = subbasin_attributes.at[72781,'RainGarden_%PervArea_Treated']  
            lid_sub_72781[3].from_pervious = subbasin_attributes.at[72781,'Bioretention_%PervArea_Treated']  
            lid_sub_72781[4].from_pervious = subbasin_attributes.at[72781,'GrassSwale_%PervArea_Treated']  

            lid_sub_72731[2].from_pervious = subbasin_attributes.at[72731,'RainGarden_%PervArea_Treated']  
            lid_sub_72731[3].from_pervious = subbasin_attributes.at[72731,'Bioretention_%PervArea_Treated']  
            lid_sub_72731[4].from_pervious = subbasin_attributes.at[72731,'GrassSwale_%PervArea_Treated']  

            lid_sub_72951[2].from_pervious = subbasin_attributes.at[72951,'RainGarden_%PervArea_Treated']  
            lid_sub_72951[3].from_pervious = subbasin_attributes.at[72951,'Bioretention_%PervArea_Treated']  
            lid_sub_72951[4].from_pervious = subbasin_attributes.at[72951,'GrassSwale_%PervArea_Treated']  

            lid_sub_73211[2].from_pervious = subbasin_attributes.at[73211,'RainGarden_%PervArea_Treated']  
            lid_sub_73211[3].from_pervious = subbasin_attributes.at[73211,'Bioretention_%PervArea_Treated']  
            lid_sub_73211[4].from_pervious = subbasin_attributes.at[73211,'GrassSwale_%PervArea_Treated']  

            lid_sub_73181[2].from_pervious = subbasin_attributes.at[73181,'RainGarden_%PervArea_Treated']  
            lid_sub_73181[3].from_pervious = subbasin_attributes.at[73181,'Bioretention_%PervArea_Treated']  
            lid_sub_73181[4].from_pervious = subbasin_attributes.at[73181,'GrassSwale_%PervArea_Treated']  

            lid_sub_73171[2].from_pervious = subbasin_attributes.at[73171,'RainGarden_%PervArea_Treated']  
            lid_sub_73171[3].from_pervious = subbasin_attributes.at[73171,'Bioretention_%PervArea_Treated']  
            lid_sub_73171[4].from_pervious = subbasin_attributes.at[73171,'GrassSwale_%PervArea_Treated']  

            lid_sub_73121[2].from_pervious = subbasin_attributes.at[73121,'RainGarden_%PervArea_Treated']  
            lid_sub_73121[3].from_pervious = subbasin_attributes.at[73121,'Bioretention_%PervArea_Treated']  
            lid_sub_73121[4].from_pervious = subbasin_attributes.at[73121,'GrassSwale_%PervArea_Treated']  

            lid_sub_73201[2].from_pervious = subbasin_attributes.at[73201,'RainGarden_%PervArea_Treated']  
            lid_sub_73201[3].from_pervious = subbasin_attributes.at[73201,'Bioretention_%PervArea_Treated']  
            lid_sub_73201[4].from_pervious = subbasin_attributes.at[73201,'GrassSwale_%PervArea_Treated']  

            lid_sub_73301[2].from_pervious = subbasin_attributes.at[73301,'RainGarden_%PervArea_Treated']  
            lid_sub_73301[3].from_pervious = subbasin_attributes.at[73301,'Bioretention_%PervArea_Treated']  
            lid_sub_73301[4].from_pervious = subbasin_attributes.at[73301,'GrassSwale_%PervArea_Treated']  

            lid_sub_73321[2].from_pervious = subbasin_attributes.at[73321,'RainGarden_%PervArea_Treated']  
            lid_sub_73321[3].from_pervious = subbasin_attributes.at[73321,'Bioretention_%PervArea_Treated']  
            lid_sub_73321[4].from_pervious = subbasin_attributes.at[73321,'GrassSwale_%PervArea_Treated']  

            lid_sub_73391[2].from_pervious = subbasin_attributes.at[73391,'RainGarden_%PervArea_Treated']  
            lid_sub_73391[3].from_pervious = subbasin_attributes.at[73391,'Bioretention_%PervArea_Treated']  
            lid_sub_73391[4].from_pervious = subbasin_attributes.at[73391,'GrassSwale_%PervArea_Treated']  

            lid_sub_73461[2].from_pervious = subbasin_attributes.at[73391,'RainGarden_%PervArea_Treated']  
            lid_sub_73461[3].from_pervious = subbasin_attributes.at[73391,'Bioretention_%PervArea_Treated']  
            lid_sub_73461[4].from_pervious = subbasin_attributes.at[73391,'GrassSwale_%PervArea_Treated']  

            lid_sub_73551[2].from_pervious = subbasin_attributes.at[73551,'RainGarden_%PervArea_Treated']  
            lid_sub_73551[3].from_pervious = subbasin_attributes.at[73551,'Bioretention_%PervArea_Treated']  
            lid_sub_73551[4].from_pervious = subbasin_attributes.at[73551,'GrassSwale_%PervArea_Treated']  

            lid_sub_73421[2].from_pervious = subbasin_attributes.at[73421,'RainGarden_%PervArea_Treated']  
            lid_sub_73421[3].from_pervious = subbasin_attributes.at[73421,'Bioretention_%PervArea_Treated']  
            lid_sub_73421[4].from_pervious = subbasin_attributes.at[73421,'GrassSwale_%PervArea_Treated']  

            lid_sub_73531[2].from_pervious = subbasin_attributes.at[73531,'RainGarden_%PervArea_Treated']  
            lid_sub_73531[3].from_pervious = subbasin_attributes.at[73531,'Bioretention_%PervArea_Treated']  
            lid_sub_73531[4].from_pervious = subbasin_attributes.at[73531,'GrassSwale_%PervArea_Treated']  

            lid_sub_73571[2].from_pervious = subbasin_attributes.at[73571,'RainGarden_%PervArea_Treated']  
            lid_sub_73571[3].from_pervious = subbasin_attributes.at[73571,'Bioretention_%PervArea_Treated']  
            lid_sub_73571[4].from_pervious = subbasin_attributes.at[73571,'GrassSwale_%PervArea_Treated']  



        # Run the simulation
            for step in sim:
                pass
            sim.report()
            sim.close()
            
            
        # Get Outfall Peak Flow
        
        reportColumns = [str(i) for i in np.arange(0,15)]
        rpt_1405 = pd.read_csv(r'C:\Users\R. Seth Herbst\Desktop\SWMM Programs and Files\PYTHON CSV files for MC\GA Parts\SWMM_Results\member{}.txt'.format(file), skiprows = 55169, nrows = 1, sep="   |    ", names = reportColumns, header = None, engine = 'python')
        rpt_1405.dropna(axis = 1, inplace = True); # This might be redundant
        # Check df.head before renaming the columns
        rpt_1405.rename(columns = {'0':'Outfall Node','6':'Flow Freq %','7':'Avg Flow (CFS)','8':'Max Flow (CFS)','10':'Total Volume (10^6 Gallons)','12':'Total Nitrogen Load (lbs)','14':'Total Phosphorous Load (lbs)'},inplace = True)
        
        peak_flows[file] = rpt_1405.at[0,"Max Flow (CFS)"]
        
        
        # Get Total Runoff Score
        
        reportColumns = [str(i) for i in np.arange(0,15)]
        rpt_total_runoff = pd.read_csv(r'C:\Users\R. Seth Herbst\Desktop\SWMM Programs and Files\PYTHON CSV files for MC\GA Parts\SWMM_Results\member{}.txt'.format(file), skiprows = 54379, nrows = 1, sep="   |    ", names = reportColumns, header = None, engine = 'python')
        rpt_total_runoff.dropna(axis = 1, inplace = True); # This might be redundant
        # Check df.head before renaming the columns
        rpt_total_runoff.rename(columns = {'0':'','2': 'Surface Runoff (AF)', '5':'Depth (in)'},inplace = True)
        
        total_runoff_volumes[file] = rpt_total_runoff.at[0,"Surface Runoff (AF)"]
        



In [12]:
swmm_model()

In [15]:
population_fitness_metrics = Member_Fitness_Calculator()

population_fitness_metrics

,Population_Members,Peak_Flow (CFS),Total_Runoff_Volume (AF),Total_Costs $,Social_Context_Score
0,member0,307.21,45.980,8.940737e+06,0.000219
1,member1,308.67,46.358,7.980429e+06,0.000263
2,member2,308.54,46.189,8.300608e+06,0.000244
3,member3,306.34,46.036,8.823641e+06,0.000225
4,member4,305.60,46.141,8.337712e+06,0.000263
5,member5,306.63,46.266,8.027175e+06,0.000296


In [16]:
Population_Elites()

{'Peak Flow': 'member4',
 'Total Runoff': 'member0',
 'Total Costs': 'member1',
 'Social Context': 'member0'}